In [ ]:
from asyncio.log import logger
from re import I
import os, sys
import sys
import asyncio
from injector import V


from src.common.azure_clients.event_grid_client_manager import EventGridClientManager, EventGridNotifier
from src.common.models.orchestrator_result import OrchestratorResult
from src.silver.context.silver_parser import SilverParser
from src.common.spark.spark_service import SparkService
from src.common.config.config_manager import ConfigManager
from src.common.enums.etl_layers import ETLLayer
from src.common.factories.orchestrator_factory import OrchestratorFactory
import src.silver.init.silver_init 

In [ ]:
def send_event_grid_notification(config: ConfigManager, orchestrator_result: OrchestratorResult):
        """
        Tworzy i wysyła powiadomienie Event Grid na podstawie OrchestratorResult.
        """
        logger.info(f"Sending Event Grid notification for correlation ID: {orchestrator_result.correlation_id}")
        

        payload = {
            "layer": orchestrator_result.etl_layer.value,
            "env": orchestrator_result.env.value,
            "status": orchestrator_result.status,
            "message_date": orchestrator_result.timestamp.isoformat(),
            "correlation_id": orchestrator_result.correlation_id,
            "manifest": "/gold/manifest/dev.manifest.json",
            "summary_processing_uri": orchestrator_result.summary_url,
            "duration_in_ms": orchestrator_result.duration_in_ms
        }

        try:
            notifier = EventGridNotifier(config.get("EVENT_GRID_ENDPOINT"), config.get("EVENT_GRID_KEY"))
            notifier.send_notification(orchestrator_result.etl_layer.value, 
                                       f"{orchestrator_result.etl_layer.value.capitalize()}ProcessCompleted",
                                       payload,
                                       orchestrator_result.correlation_id)
            
            logger.info("Event Grid notification sent successfully.")
        except Exception as e:
            logger.error(f"Failed to send Event Grid notification: {e}", exc_info=True)
            

In [21]:
dbutils.widgets.text("summary_json", "")
dbutils.widgets.text("manifest_json", "")

manifest_json = dbutils.widgets.get("manifest_json")
summary_json = dbutils.widgets.get("summary_json")


config = ConfigManager()
spark = SparkService(config)
spark.start_local()

context = SilverParser(config).parse(manifest_json, summary_json)

orchestrator = OrchestratorFactory.get_instance(ETLLayer.SILVER, spark=spark, config=config)
result = asyncio.run(orchestrator.execute(context))
send_event_grid_notification(config=config, orchestrator_result=result)

Passed unknown parameter: TEST
Input notebook does not contain a cell with tag 'parameters'
Executing: 100%|██████████| 2/2 [00:04<00:00,  2.07s/cell]


{'cells': [{'id': 'e22ec407',
   'cell_type': 'code',
   'metadata': {'tags': ['injected-parameters'],
    'papermill': {'exception': False,
     'start_time': '2025-10-15T08:26:01.822042',
     'end_time': '2025-10-15T08:26:01.834106',
     'duration': 0.012064,
     'status': 'completed'},
    'execution': {'iopub.status.busy': '2025-10-15T08:26:01.825022Z',
     'iopub.execute_input': '2025-10-15T08:26:01.826020Z',
     'shell.execute_reply': '2025-10-15T08:26:01.832088Z',
     'iopub.status.idle': '2025-10-15T08:26:01.833100Z'}},
   'execution_count': 1,
   'source': '# Parameters\nTEST = "I\'m here 2"\n',
   'outputs': []},
  {'cell_type': 'code',
   'execution_count': 2,
   'metadata': {'tags': [],
    'papermill': {'exception': False,
     'start_time': '2025-10-15T08:26:01.836101',
     'end_time': '2025-10-15T08:26:01.844165',
     'duration': 0.008064,
     'status': 'completed'},
    'execution': {'iopub.status.busy': '2025-10-15T08:26:01.839141Z',
     'iopub.execute_input'

In [14]:
%run ../../../src/azure_databricks/notebooks/test.ipynb

I'm here
